In [19]:
# put all imports in one sell to avoid running all cells to continue programming
%matplotlib inline
import matplotlib.pyplot as plt
import sys  
import pandas as pd
sys.path.insert(0, 'C:/Users/h.karimi/Documents/pprojects/housing_hw/src/features')
sys.path.insert(0, 'C:/Users/h.karimi/Documents/pprojects/housing_hw/src/data')
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from load_data import internal_load
from columnsSpliter import get_columns
import math
data = internal_load("C:/Users/h.karimi/Documents/pprojects/housing_hw/data/interim/htt_with_lat_long.csv")
original_data = internal_load("C:/Users/h.karimi/Documents/pprojects/housing_hw/data/external/test.csv")

In [20]:
# removing duplicate 
def remove_duplicate (data):
    data_copy = data.copy()
    data_new = data_copy.drop_duplicates()
    return(data_new)

# decapitalizing
def decapitalizing(data,string_columns,categorical_attribs):
    data_copy = data.copy()
    for column in string_columns: # decapitalizing the data
        data_copy[column] = data_copy[column].str.lower().copy()
        
    for col in string_columns: # decapitalizing the metadata
        ca_copy = categorical_attribs[col].copy()
        categorical_attribs[col] = [x.lower() for x in ca_copy]
    
    return data_copy

def strc_errors_handler(data,cat_columns, correction = True):
    data_copy = data.copy()
    strc_errors_printer.err_cols = {}
    for column in cat_columns:
        i = 0
        while i < len(data_copy[column]):
            case = data_copy.at[i,column]
            if case not in categorical_attribs[column] and not pd.isna(case):
                if column not in strc_errors_printer.err_cols.keys():
                    strc_errors_printer.err_cols[column] = set()
                strc_errors_printer.err_cols[column].add(case)
                if correction:
                    data_copy.drop(i,inplace = True)
            i += 1
    return data_copy

def correct_current_typos(data):
    
    data_copy = data.copy()
    data_copy.replace('twnhs','twnhsI')
    data_copy.replace('brk cmn','brkcomm',inplace = True)
    data_copy.replace('cmentbd','cemntbd',inplace = True)
    data_copy.replace('wd shng','wdshing',inplace = True)
    
    return data_copy

def na_handler(data,numerical_columns, numcat_columns,string_columns):
    
    data_copy_num = data[numerical_columns].copy()
    data_copy_numcat = data[numcat_columns].copy()
    data_copy_string = data[string_columns].copy()
    
    data_copy_num.interpolate(inplace = True)
    for col in data_copy_num:
        i = 0
        if (pd.isna(data_copy_num.at[0,col])):
            while pd.isna(data_copy_num.at[i,col]):
#                 print(i ,' yes ', col)
                i += 1
            for j in range(i):
#                 print(i ,' yes again ', col)
                data_copy_num.at[j,col] = data_copy_num.at[j+1,col]
        data_copy_numcat.interpolate(method = 'pad', inplace = True)
    data_copy_string.fillna(value = 'na',inplace = True)
    
    
    
    return pd.concat([data_copy_num,data_copy_numcat,data_copy_string], axis=1)
        


In [21]:
#packing all above function
def data_cleaning(data):
    strCols = get_columns('str')
    nmcCols = get_columns('nmc')
    catCols = get_columns('cat')
    numCols = get_columns('num')
    data_copy = data.copy()
    return na_handler(correct_current_typos(decapitalizing(remove_duplicate(data_copy),strCols,catCols)),numCols,nmcCols,strCols)


In [22]:
data_copy = data_cleaning(data)


In [25]:
data_copy.to_csv(r'C:\Users\h.karimi\Documents\pprojects\housing_hw\data\interim\housing_train_cleaned.csv')
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1168 entries, 0 to 1167
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotFrontage    1168 non-null   float64
 1   LotArea        1168 non-null   int64  
 2   YearBuilt      1168 non-null   int64  
 3   YearRemodAdd   1168 non-null   int64  
 4   MasVnrArea     1168 non-null   float64
 5   BsmtFinSF1     1168 non-null   int64  
 6   BsmtFinSF2     1168 non-null   int64  
 7   BsmtUnfSF      1168 non-null   int64  
 8   TotalBsmtSF    1168 non-null   int64  
 9   1stFlrSF       1168 non-null   int64  
 10  2ndFlrSF       1168 non-null   int64  
 11  LowQualFinSF   1168 non-null   int64  
 12  GrLivArea      1168 non-null   int64  
 13  BsmtFullBath   1168 non-null   int64  
 14  BsmtHalfBath   1168 non-null   int64  
 15  FullBath       1168 non-null   int64  
 16  HalfBath       1168 non-null   int64  
 17  BedroomAbvGr   1168 non-null   int64  
 18  KitchenA

In [24]:
# with pd.option_context('display.max_rows', 5, 'display.max_columns', None):
#     display(data_copy)